# Réalisation du projet en autonomie

In [1]:
#Chargement des librairies
#Chargement des images
#Création d'un ensemble de test et d'apprentissage, affichage d'exemple

Commenter les opérations de chargement et de création des bases

## Structure imposée

In [4]:
# Création de la structure
#Apprentissage de la structure
#MEsure du temsp d'apprentissage
#Etude des courbes d'apprentissage

Commentaire sur l'apprentissage

In [5]:
#Etude des résultats globaux et par classe

Commentaire sur l'efficience du modèle et conclusion

## Structure libre mais à complexité contrôlée

In [4]:
# Création de la structure
#Apprentissage de la structure
#MEsure du temsp d'apprentissage
#Etude des courbes d'apprentissage

Commentaire sur l'apprentissage et sur les choix faits pour la structure

In [5]:
#Etude des résultats globaux et par classe

Commentaire sur l'efficience du modèle, comparaison avec le premier modèle et conclusion

## Structure libre mais à discrimination optimale

In [4]:
# Création de la structure
#Apprentissage de la structure
model.save('my_modele')
#MEsure du temsp d'apprentissage
#Etude des courbes d'apprentissage

Commentaire sur l'apprentissage et sur les choix faits pour la structure

In [6]:
#Etude des résultats globaux et par classe

Commentaire sur l'efficience du modèle, comparaison avec les premiers modèles et conclusion

## Création d'une structure autoencodeur

In [ ]:
# codeur correspond à la structure imposée
#décodeur
#autoencoder_dr = Model(inputs = encoder.inputs, outputs = decoder(encoder.outputs))
# Apprentissage avec comme fonction de cout mse

Commentaire sur l'apprentissage et sur les choix faits pour la structure

In [8]:
#Projection d'images de la base
#autoencoder_dr.predict()

Commentaire sur la qualité d'approximation de votre autoencodeur évaluée par traitement de différentes images de la base

## Transfert learning du codeur

In [ ]:
# récupération du codeur de l'autoencodeur
# Ajout de couche de prise de décision
# Apprentissage en bloquant les poids du codeur
for layer in encoder.layers:
    layer.trainable = False

Commentaire sur l'apprentissage et sur la structure

In [9]:
#Etude des résultats globaux et par classe

Commentaire sur l'efficience du modèle, comparaison avec le premier modèle et conclusion

## Influence du bruit

In [10]:
def ma_fonction(Data):
    Data_b = Data.copy()
    for nb in range(0,len(Data)):
        Data_b[nb] = Data[nb]+(np.random.rand(Data[nb].shape[0],Data[nb].shape[1],Data[nb].shape[2])*0.7)
    Data_b = np.clip(Data_b,0.,1.)
    return Data_b

Expliciter ce que réalise cette fonction et indiquant l'opération réalisée par chaque ligne

In [ ]:
#Test sur la meilleure structure 
my_model = keras.models.load_model('modele')
X_test_b = ma_fonction(X_test)
test_loss, test_acc = model.evaluate(X_test_b, y_test)

Commenter les opérations, évaluer les résultats par rapport à ceux obtenus sur X_test.

## Autoencodeur contre le bruit